<h1>Newton's method (system of non-linear equations)</h1>

<h2>Code</h2>

In [11]:
#R.<x, y> = RDF[]

PREC = 6

var('x, y, a, k')

def set_prec(T):
    return [[v.n(digits=PREC) for v in row] for row in T]

def ev(F, X, vars=None):
    if not vars:
        vars = (str(v) for v in reduce(union, (f.variables() for f in F)))
    return F(**dict(zip(vars, X)))

def find_initial_point(F, G, step=0.5, eps=0.1, verbose=False):
    if verbose:
        html('<h3>Searching for possible candidates for the initial $\mathbf{x}_0$</h3>')
        html('$(x, y)$\t\t\t\t$\mathbf{F}(x,y)$\n')
    candidate = {}
    for X in CartesianProduct(*[srange(I[0], I[1], step, include_endpoint=True, universe=RDF)
                                for I in G]):
        val = vector(ev(F, X))
        if all((abs(val[i]) <= eps for i in range(len(F)))):
            if verbose:
                html('$%s$\t\t$%s$' % ([v.n(digits=PREC) for v in X],
                                       [v.n(digits=PREC) for v in ev(F, X)]))
            if len(candidate) > 0:
                if abs(val) >= abs(candidate['val']): # abs calculates L_2 metric
                    continue
            candidate['val'] = val
            candidate['point'] = vector(X)
            if verbose:
                html(r'(candidate with $\left\Vert %s\right\Vert $)' % abs(val))
    if verbose:
        html('')
    if len(candidate) == 0:
        return None
    else:
        return candidate['point']
    
def find_Xn(X, DX, F, n_iter=0, max_iter=30, eps=10e-5, history=False):
    if history is not False:
        history.append([n_iter, X, F(x=X[0], y=X[1])])
    if n_iter == max_iter:
#        print('WARNING: maximum number of iterations reached')
        return None
    else:
        Xn = X + ev(DX, X)
        if all(abs(x_i) <= eps for x_i in Xn - X):
            if history is not False:
                history.append([n_iter+1, Xn, ev(F, Xn)])
            return vector(Xn)
        else:
            return find_Xn(Xn, DX, F, n_iter+1, max_iter=max_iter, eps=eps, history=history)

def solve_snle_newton(F, G, step=0.5, eps_x0=0.1, eps_newton=10e-5, verbose=False):
    html('<h2>Solving SNLE on %s</h2>' % G)
    X0 = find_initial_point(F, G, step, eps_x0, verbose=verbose)
    if verbose:
        html('Found initial $\mathbf{x}_0 = %s$' % X0)
    # TODO: generalize
    M = [
        matrix([[-F[0], diff(F[0], y)],
                [-F[1], diff(F[1], y)]]),
        matrix([[diff(F[0], x), -F[0]],
                [diff(F[1], x), -F[1]]])
        ]
    denom = det(matrix([[diff(F[0], x), diff(F[0], y)],
                        [diff(F[1], x), diff(F[1], y)]]))
    DX = vector([det(m) / denom for m in M])
    #if verbose:
    #    for D in DX:
    #        show(D)
    if verbose:
        html('<h3>Searching for solutions using Newton\'s method</h3>')
        history = []
    else:
        history = False
    solution = find_Xn(X0, DX, F, history=history)
    if verbose:
        html.table(set_prec(history), header=["$n$", "$(x,y)$", "$\mathbf{F}(x,y)$"])
        html('Found solution: %s' % solution)
    return solution
    
def make_partition(G, rank):
    Ps = []
    for I in G:
        P1 = srange(I[0], I[1], step=rank, include_endpoint=True)
        P_int = []
        for i in range(len(P1)-1):
            P_int.append((P1[i], P1[i+1]))
        Ps.append(P_int)   
    partition = CartesianProduct(*Ps).list()
    return partition
    
def find_all_roots(F, G, rank, step=0.5, eps_x0=0.1, eps_newton=10e-5, verbose=False):
    partition = make_partition(G, rank)
    roots = []
    for square in partition:
        root = solve_snle_newton(F, square, step, eps_x0, eps_newton, verbose)
        if root:
            roots.append(root)
    return roots

<h2>Data</h2>

In [20]:
F = vector([sin(x + k*y) - x + y^2, (y+0.1)^2 + x^2 - a])
G = [[0, 2], [0, 2]]
A = srange(0.6, 1.1, 0.1, universe=RDF, include_endpoint=True)
K = srange(-0.5, -0.1, 0.1, universe=RDF, include_endpoint=True)
colors=('blue', 'green')

In [6]:
G = [[-2, 2], [-2, 2]]
A = srange(0, 1, 0.001, universe=RDF, include_endpoint=True)
K = srange(-4, 6, 0.1, universe=RDF, include_endpoint=True)

#def plot_F_interact(find_solution=checkbox(label='Find solution', default=False),
@interact
def plot_F_interact(a=slider(A, label='$a$', default=0.4),
                    k=slider(K, label='$k$', default=-0.8)):
    F_p = F(a=a, k=k)
    html('<h2>Plot of</h2>')
    show(F_p)
    #graph = sum(plot3d(F_p[i], *G, color=colors[i], opacity=0.6) for i in range(len(F)))
    graph = sum(implicit_plot(F_p[i]==0, *G, color=colors[i]) for i in range(len(F)))
    
    graph.show()

<html><!--notruncate-->
        <div padding=6 id="div-interact-6">
          <table width=800px height=20px bgcolor="#c5c5c5" cellpadding=15>
            <tr>
              <td bgcolor="#f9f9f9" valign=top align=left>
            <table>
              <tr><td colspan=3><table><tr><td align=right><font color="black">$a$ </font></td><td>
        <table>
          <tr>
            <td>
              <div id="slider-a-6" style="margin:0px; margin-left: 1.0em; margin-right: 1.0em; width: 15.0em;"></div>
            </td>
            
            <td>
              <font color="black" id="slider-a-6-lbl"></font>
            </td>
          </tr>
        </table><script>
    (function() {
        var values = ["0.0","0.002","0.004","0.006","0.008","0.010000000000000002","0.012000000000000004","0.014000000000000005","0.016000000000000007","0.01800000000000001","0.02000000000000001","0.022000000000000013","0.024000000000000014","0.026000000000000016","0.028000000000000018","0.03000000000000002

In [18]:
A = srange(0.6, 1.1, 0.1, universe=RDF, include_endpoint=True)
K = srange(-0.5, -0.1, 0.1, universe=RDF, include_endpoint=True)
G = [[-2, 2], [-2, 2]]

@interact
def solve_snle_newton_interact(a=slider(A, label='$a$'),
                               k=slider(K, label='$k$'),
                               verbose=checkbox(default=True)):
    print find_all_roots(F(a=a, k=k), G, rank=2, step=0.1, verbose=verbose)

<html><!--notruncate-->
        <div padding=6 id="div-interact-18">
          <table width=800px height=20px bgcolor="#c5c5c5" cellpadding=15>
            <tr>
              <td bgcolor="#f9f9f9" valign=top align=left>
            <table>
              <tr><td colspan=3><table><tr><td align=right><font color="black">$a$ </font></td><td>
        <table>
          <tr>
            <td>
              <div id="slider-a-18" style="margin:0px; margin-left: 1.0em; margin-right: 1.0em; width: 15.0em;"></div>
            </td>
            
            <td>
              <font color="black" id="slider-a-18-lbl"></font>
            </td>
          </tr>
        </table><script>
    (function() {
        var values = ["0.6","0.7","0.7999999999999999","0.8999999999999999","0.9999999999999999","1.1"];
        setTimeout(function() {
            $('#slider-a-18').slider({
                step: 1,
                min: 0,
                max: 5,
                value: 0,
                change: functi